In [31]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import Levenshtein as lev

In [38]:
lev.ratio("Joseph Walden Johnson Jr.", "Joseph Johnson")

0.717948717948718

In [30]:
SequenceMatcher(None, "Joseph Walden Johnson Jr.", "Joseph Johnson").ratio()
"Joseph Johnson" in "Joseph Walden Johnson Jr."

False

In [3]:
data3 = pd.read_csv("killedbypolice.csv")
data2 = pd.read_csv("MPVDatasetDownload.csv")
data1 = pd.read_csv("https://raw.githubusercontent.com/washingtonpost/data-police-shootings/master/fatal-police-shootings-data.csv")

In [4]:
data1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [5]:
data2.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Criminal Charges?,Link to news article or photo of official document,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Unnamed: 24
0,Saheed Vassell,34,Male,Black,NaN,4/4/18,NaN,Brookyn,NY,NaN,...,No known charges,NaN,Yes,Unclear,metal pipe,attack,Not fleeing,No,3577.0,NaN
1,Timothy Orlando Wyatt,41,Male,Black,http://www.fatalencounters.org/wp-content/uplo...,4/1/18,E 28th Ave & N 15th St,Tampa,FL,33605.0,...,No known charges,http://www.wfla.com/news/hillsborough-county/p...,No,Allegedly Armed,gun,attack,Car,No,3564.0,NaN
2,Joseph Walden Johnson Jr.,24,Male,White,http://www.fatalencounters.org/wp-content/uplo...,4/1/18,Railroad Avenue,Covington,LA,70433.0,...,No known charges,http://www.nola.com/crime/index.ssf/2018/04/st...,No,Allegedly Armed,gun,attack,Car,No,3563.0,NaN
3,Bobby Hinton,60,Male,Unknown race,NaN,4/1/18,Magnolia Street,New Llano,LA,71461.0,...,No known charges,http://www.klfy.com/top-stories/deputy-involve...,No,Allegedly Armed,gun,attack,Not fleeing,No,3562.0,NaN
4,Christopher Lamarr Hall,34,Male,Black,NaN,4/1/18,4100 SW Clyde Park Ave,Wyoming,MI,49509.0,...,No known charges,http://www.mlive.com/news/jackson/index.ssf/20...,Yes,Allegedly Armed,gun,attack,Not fleeing,No,3566.0,NaN


In [6]:
data3.head()

,age,date,gender,manner of death,name,race,state,month,year
0,25,"April 21, 2018",M,shot,Terrance Carlton,NaN,TN,April,2018
1,48,"April 20, 2018",M,shot,James Bauduy,NaN,FL,April,2018
2,40,"April 19, 2018",M,shot,Charles Whitley,NaN,KY,April,2018
3,45,"April 18, 2018",M,shot,William Ray Simcoe Sr.,W,MO,April,2018
4,48,"April 18, 2018",M,shot,David Gino Teneyuque,NaN,MI,April,2018


In [7]:
data1names = list(data1["name"])
data2names = list(data2["Victim's name"])
data3names = list(data3["name"])

### Dates After 2015

In [8]:
dates2 = list(data2["Date of Incident (month/day/year)"])
months2 = [int(date[:date.find("/")]) for date in dates2]
years2 = [int("20"+date[date.rfind("/")+1:]) for date in dates2]
data2["month"] = np.array(months2)
data2["year"] = np.array(years2)
past20152 = data2.loc[((data2["year"]>=2015) & (data2["year"]<2018)) | ((data2["month"]<4)&(data2["year"]==2018))]["Victim's name"]

In [9]:
dates3 = list(data3["date"])
months3 = [date.split()[0] for date in dates3]
years3 = [int(date.split()[2]) for date in dates3]
data3["month"] = np.array(months3)
data3["year"] = np.array(years3)
past20153 = data3.loc[((data3["year"]>=2015) & (data3["year"]<2018)) | (((data3["month"] == "January") | (data3["month"] == "February") | (data3["month"]== "March")) & (data3["year"]==2018))]["name"]

In [12]:
wsp = data1names
mpv = list(past20152)
kbp = list(past20153)

In [55]:
# cleaning data
newmpv = []
for name in mpv:
    if name != "Name withheld by police":
        newmpv.append(name)
mpv = newmpv
newwsp = []
for name in wsp:
    if name.lower() != "tk tk":
        newwsp.append(name)
wsp = newwsp

In [50]:
print("Number of names in the Washington Post dataset:", "\t", len(wsp))
print("Number of names in the Mapping Police Violence dataset:", "\t", len(mpv))
print("Number of names in the Killed By Police dataset:", "\t", len(kbp))

Number of names in the Washington Post dataset: 	 3208
Number of names in the Mapping Police Violence dataset: 	 3702
Number of names in the Killed By Police dataset: 	 3148


## Common Names Across All 3 Datasets

In [58]:
list12 = []
for name1 in wsp:
    for name2 in mpv:
        if lev.ratio(name1, name2) > 0.7:
            list12.append(name2)
            break

In [59]:
ls = []
for name in list12:
    for name3 in kbp:
        if lev.ratio(name, name3) > 0.7:
            ls.append(name3)
            break
len(ls)

2739

## Common Names Across Killed By Police and Washington Post

In [56]:
list12 = []
list12missing = []
for name1 in wsp:
    found = False
    for name2 in kbp:
        if lev.ratio(name1, name2) > 0.7:
            list12.append(name1)
            found = True
            break
    if not found:
        list12missing.append(name1)
len(list12)

2786

### Missing names from Killed by police that are in washington post

In [57]:
list12missing

['Omarr Julian Maximillian Jackson',
 'Edward Donnell Bright',
 'Dewayne Deshawn Ward',
 'Desmond Luster',
 'Howard Brent Means Jr.',
 'Alexander Phillip Long',
 'Charly Leundeu Keunang',
 'Hung Trieu',
 'Mya Hall',
 'Frank Shephard',
 'Billy Joe Patrick',
 'Nadir Soofi',
 'Dennis  Fiel',
 'James Strong',
 'Lorenzo Garza',
 'Jesus Quezada Gomez',
 'Raymond Peralta-Lantigua',
 'Victor Emanuel Larosa',
 'Paul Castaway',
 'Adam Schneider',
 'Kyle Lambrose',
 'Jose Ramon Damiani Jr',
 'Angelo Delano Perry',
 'India Kager',
 'Bruce Santino',
 'Rory Lynn Gunderman',
 'Tina Money',
 'Norma Angelica Guzman',
 'Leslie Portis',
 'Martin  Ryans Jr.',
 'Roger D.  Hall',
 'Ryan Christopher Rodriguez',
 'Tony Berry',
 'Shane Whitehead',
 'Tashfeen Malik',
 'Syed Farook',
 'Bettie Jones',
 'Quintonio LeGrier',
 'Gayneaux Paul Trahan',
 'Keith Childress',
 'Lonnie Haskell Powers',
 'Clarence Layfield',
 'Shawn Henry Allen Dickens',
 'Jose Quintanilla',
 'Oscar Reyes',
 'Che Taylor',
 'Kisha Michael',


## Common Names Across Mapping Police Violence and Washington Post

In [45]:
list13 = []
for name1 in wsp:
    for name2 in mpv:
        if lev.ratio(name1, name2) > 0.7:
            list13.append(name2)
            break
len(list13)

3120

## Common Names Across Killed By Police and Mapping Police Violence

In [46]:
list23 = []
for name1 in kbp:
    for name2 in mpv:
        if lev.ratio(name1, name2) > 0.7:
            list23.append(name2)
            break
len(list23)

2983

## Other Tries at Analysis

In [34]:
# creating columns for first and last name in each data set
firstlast = []
for name in data2names:
    firstname = name[0:name.find(" ")]
    lastname = name[name.rfind(" ")+1: ]
    if len(lastname) <= 3:
        name = name[0:name.rfind(" ")]
        lastname = name[name.rfind(" "):]
    firstlast.append(firstname + " " + lastname)
data2["first and last name"] = firstlast
data2

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Link to news article or photo of official document,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Unnamed: 24,first and last name
0,Saheed Vassell,34,Male,Black,NaN,2018-04-04,NaN,Brookyn,NY,NaN,...,NaN,Yes,Unclear,metal pipe,attack,Not fleeing,No,3577.0,NaN,Saheed Vassell
1,Timothy Orlando Wyatt,41,Male,Black,http://www.fatalencounters.org/wp-content/uplo...,2018-04-01,E 28th Ave & N 15th St,Tampa,FL,33605.0,...,http://www.wfla.com/news/hillsborough-county/p...,No,Allegedly Armed,gun,attack,Car,No,3564.0,NaN,Timothy Wyatt
2,Joseph Walden Johnson Jr.,24,Male,White,http://www.fatalencounters.org/wp-content/uplo...,2018-04-01,Railroad Avenue,Covington,LA,70433.0,...,http://www.nola.com/crime/index.ssf/2018/04/st...,No,Allegedly Armed,gun,attack,Car,No,3563.0,NaN,Joseph Johnson
3,Bobby Hinton,60,Male,Unknown race,NaN,2018-04-01,Magnolia Street,New Llano,LA,71461.0,...,http://www.klfy.com/top-stories/deputy-involve...,No,Allegedly Armed,gun,attack,Not fleeing,No,3562.0,NaN,Bobby Hinton
4,Christopher Lamarr Hall,34,Male,Black,NaN,2018-04-01,4100 SW Clyde Park Ave,Wyoming,MI,49509.0,...,http://www.mlive.com/news/jackson/index.ssf/20...,Yes,Allegedly Armed,gun,attack,Not fleeing,No,3566.0,NaN,Christopher Hall
5,Cresencio Rodriguez,33,Male,Unknown race,NaN,2018-04-01,505 TX-1604 Loop,San Antonio,TX,78253.0,...,https://www.ksat.com/news/deputies-shoot-kill-...,No,Allegedly Armed,gun,other,Not fleeing,No,3561.0,NaN,Cresencio Rodriguez
6,Brian Bellamy,41,Male,Black,http://www.fatalencounters.org/wp-content/uplo...,2018-04-01,757 Fox Chase Ln,Greenville,NC,28590.0,...,http://wcti12.com/news/local-crime/officer-inv...,No,Allegedly Armed,knife,other,Not fleeing,No,3565.0,NaN,Brian Bellamy
7,Luis Yair Alvarez,22,Male,Hispanic,NaN,2018-04-01,2600 Florencia Avenue,Brownsville,TX,78526.0,...,http://www.brownsvilleherald.com/news/local/te...,Yes,Allegedly Armed,knife,other,Not fleeing,No,3572.0,NaN,Luis Alvarez
8,Name withheld by police,Unknown,Male,White,NaN,2018-04-01,208 Spring Street,Belton,MO,64012.0,...,http://fox4kc.com/2018/04/02/man-fatally-shot-...,Drug or alcohol use,Allegedly Armed,knife,other,NaN,No,3573.0,NaN,Name police
9,John Wussler,58,Male,Native American,NaN,2018-03-31,E Monte Vista Rd & N 32nd St,Phoenix,AZ,85008.0,...,https://www.azcentral.com/story/news/local/pho...,No,Allegedly Armed,gun,attack,Foot,No,3571.0,NaN,John Wussler


In [40]:
firstlast = []
for name in data1names:
    firstname = name[0:name.find(" ")]
    lastname = name[name.rfind(" ")+1: ]
    if len(lastname) <= 3:
        name = name[0:name.rfind(" ")]
        lastname = name[name.rfind(" "):]
    firstlast.append(firstname + " " + lastname)
data1["first and last name"] = firstlast
data1

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,first and last name
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,Tim Elliot
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,Lewis Lembke
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,John Quintero
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,Matthew Hoffman
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,Michael Rodriguez
5,11,Kenneth Joe Brown,2015-01-04,shot,gun,18.0,M,W,Guthrie,OK,False,attack,Not fleeing,False,Kenneth Brown
6,13,Kenneth Arnold Buck,2015-01-05,shot,gun,22.0,M,H,Chandler,AZ,False,attack,Car,False,Kenneth Buck
7,15,Brock Nichols,2015-01-06,shot,gun,35.0,M,W,Assaria,KS,False,attack,Not fleeing,False,Brock Nichols
8,16,Autumn Steele,2015-01-06,shot,unarmed,34.0,F,W,Burlington,IA,False,other,Not fleeing,True,Autumn Steele
9,17,Leslie Sapp III,2015-01-06,shot,toy weapon,47.0,M,B,Knoxville,PA,False,attack,Not fleeing,False,Leslie Sapp


In [41]:
firstlast = []
for name in data3names:
    firstname = name[0:name.find(" ")]
    lastname = name[name.rfind(" ")+1: ]
    if len(lastname) <= 3:
        name = name[0:name.rfind(" ")]
        lastname = name[name.rfind(" "):]
    firstlast.append(firstname + " " + lastname)
data3["first and last name"] = firstlast
print(len(data3))
data3

4414


,age,date,gender,manner of death,name,race,state,first and last name
0,48,"April 20, 2018",M,shot,James Bauduy,NaN,FL,James Bauduy
1,40,"April 19, 2018",M,shot,Charles Whitley,NaN,KY,Charles Whitley
2,45,"April 18, 2018",M,shot,William Ray Simcoe Sr.,W,MO,William Simcoe
3,48,"April 18, 2018",M,shot,David Gino Teneyuque,NaN,MI,David Teneyuque
4,32,"April 18, 2018",M,shot,Delorean Pikyavit,I,UT,Delorean Pikyavit
5,32,"April 18, 2018",M,shot,Justin Oakes,W,GA,Justin Oakes
6,25,"April 17, 2018",M,shot,Sanchez Lowe,NaN,GA,Sanchez Lowe
7,39,"April 17, 2018",M,shot,Justin Monjay,NaN,OK,Justin Monjay
8,41,"April 17, 2018",M,shot,Lonnie Marcel Bowen,B,UT,Lonnie Bowen
9,59,"April 15, 2018",M,shot,Raymond Lyle Bell,W,WV,Raymond Bell


In [36]:
data3names = np.array(data3["name"])
print(len(data3names))
print(data3names)

4414
['James Bauduy' 'Charles Whitley' 'William Ray Simcoe Sr.' ...,
 'Eric Poore' 'Erick Balint' 'Connor Bishop Zion']


In [35]:
# name1 = data1["first and last name"]
# name2 = data2["first and last name"]
# count12 = 0
# for n1 in name1:
#     for n2 in name2:
#         if similar(n1, n2) > 0.4:
#             count12 += 1
#             break
            
# print(count12)

3282


In [37]:
name1 = data1["first and last name"]
name2 = data2["first and last name"]
count12 = 0
for n1 in name1:
    for n2 in name2:
        if n1 in n2:
            count12 += 1
            break
            
print(count12)

2852


name1 with name2 == washington post vs mpv data == 2852 matches

In [50]:
# def similar(data1names, data2names):
#     return SequenceMatcher(None, data1names,data2names).ratio()

# count = 0
# for name2 in data1names:
#     for name1 in data2names:
#         if similar(name1, name2) > 0.4:
#             count += 1
#             break
            
# print(count)

In [43]:
name3 = data3["first and last name"]
count23 = 0
for n2 in name2:
    for n3 in name3:
        if n2 in n3:
            count23 += 1
            break
            
print(count23)

# comparing MPV (data2) with KilledByPolice (data3) == 3878 matches

3878


In [47]:
name1 = data1["first and last name"]
name3 = data3["first and last name"]
countall3 = 0
both23 = []
for n2 in name2:
    for n3 in name3:
        if n2 in n3:
            both23.append(n2)
            break
print(len(both23))
# put first two for loops in a list (descending order) 

3878


In [51]:
all3 = []
for both in both23:
    for n1 in name1:
        if n1 in both:
            all3.append(n1)
            break
print(len(all3))
print(all3)

2305
['Saheed Vassell', 'Timothy Wyatt', 'Bobby Hinton', 'Christopher Hall', 'Cresencio Rodriguez', 'Brian Bellamy', 'Luis Alvarez', 'John Wussler', 'Jason Birt', 'Nicolas Sedano', 'Edward McCrae', 'Timothy Breckenridge', 'Marco Saavedra', 'Rodney Toler', 'Christopher Gatewood', 'James Decoursey', 'Jesse Kilgus', 'Aaron Ibrahem', 'Robert Shaw', 'Gerald Johns', 'Jose Gonzalez', 'Juan Romero', 'Isaac Scroggie', 'Evan Bashir', 'Nathan Castle', 'Kaitlin Demeo', 'Zander Clark', 'Nikolai Yakunin', 'Michael McGinnis', 'Kevin Robles', 'Daniel Young', 'Kevin Hall', 'Linus Phillip', 'Santiago Calderon', 'Danny Thomas', 'Duane Preciado', 'Jason Raffaeli', 'Jehad d', 'Mark Lunn', 'Chance Haegele', 'Kent Pittman', 'Scott Mielentz', 'Joel Pflum', 'Steven Nguyen', 'Michael Holliman', 'Jermaine Massey', 'Manuel Borrego', 'Stephon Clark', 'William Langfitt', 'James Vaughn', 'Bobby Lovin', 'Cameron Hall', 'Shermichael Ezeff', 'Ryan Chapman', 'Bryan Liles', 'Decynthia Clements', 'Michael Ward', 'Jontell 

"which data set is most reliable or "inclusive"? Why are certain names caught across all three and others not?"

MPV : had the most inputted names (data2)
    2305 / 5994 = 38%

KilledBy Police : had second most names (data3)
    2305 / 4414 = 52%

WashingtonPost : had least names (data1)
    2305 / 3282 = 70%